# Train a multivariat multi-output model for Solar and Windelectricity

In [ ]:
"""
Notebook to train and evaluate a stacked ensemble model for renewable electricity production forecasting.

This notebook guides the user through the following steps:
- Load preprocessed data for model training
- Defines features and target variables for renewable electricity production (wind and solar PV).
- Splits the data into training and test sets.
- Scales the features using `RobustScaler` and saves the scaler as `robust_scaler_multivariate.pkl`.
- Conducts hyperparameter tuning via randomized search for `RandomForestRegressor` and `XGBRegressor`.
- Builds a stacked ensemble model using `StackingRegressor` with `Ridge` as the final estimator.
- Wraps the ensemble model in `MultiOutputRegressor` to handle multiple targets.
- Trains the final model and saves it as `stacked_multivariate_model.pkl`.
- Evaluates the model on training and test sets using MSE, R², and adjusted R² metrics (optionally RMSE, MAE can be added).

Outputs:
- `models/robust_scaler_multivariate.pkl`: Saved RobustScaler fitted to the training data for feature transformation.
- `models/stacked_multivariate_model.pkl`: Saved trained stacked ensemble model.
- Evaluation metrics printed in the console (for individual and combined targets).

Dependencies:
- pandas, numpy, scikit-learn, xgboost, joblib, tabulate

Usage:
- Execute the notebook cells in sequence.
- Advise to keep the folderstructure to load the data and save the scaler and model. This is necessary for a functioning dashboard
"""

### Imports

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# modeling
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
# mean_absolute_error only necessary if the option is added
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib

### Load Data

In [2]:
# load already preprocessed data (weather and energy features)
df_wind_solar = pd.read_csv('data/df_clean_for_modeling_with_offshore_3y.csv', sep = ',')
# combine offshore and onshore wind contribution (needed for this model)
df_wind_solar['windpower'] = df_wind_solar.offshore_wind + df_wind_solar.onshore_wind

### Feature Selection and Train/Test-Split

In [3]:
# Specify features and target variables
features = [
    'temperature_2m_max', 'temperature_2m_min', 'temp_diff_2m',
    'apparent_temperature_max', 'apparent_temperature_min', 'apparent_temp_diff',
    'daylight_duration', 'sunshine_duration', 'precipitation_sum',
    'precipitation_hours', 'snowfall_sum', 'shortwave_radiation_sum',
    'wind_speed_10m', 'wind_direction_10m', 'wind_gusts_10m_max'
]

targets = ['windpower', 'solar_pv']

# Split the data into features (X) and targets (y)
X = df_wind_solar[features]
y = df_wind_solar[targets]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Apply and Save Scaler

In [4]:
# Apply RobustScaler to features (fit on training data only and apply on test data)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler for later use in given folder structure
joblib.dump(scaler, 'models/robust_scaler_multivariate.pkl')

['models/robust_scaler_multivariate.pkl']

### Hyperparametertuning, Train & Save the Model

In [5]:
# Perform Randomized Search for RandomForestRegressor
rf_param_dist = {
    'n_estimators': [100, 200, 500, 1000],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}
random_search_rf = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42),
                                      param_distributions=rf_param_dist,
                                      n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search_rf.fit(X_train_scaled, y_train)
best_rf = random_search_rf.best_estimator_
print("Best Parameters for RandomForestRegressor:", random_search_rf.best_params_)

# Perform Randomized Search for XGBRegressor
xgb_param_dist = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
random_search_xgb = RandomizedSearchCV(estimator=XGBRegressor(random_state=42),
                                        param_distributions=xgb_param_dist,
                                        n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search_xgb.fit(X_train_scaled, y_train)
best_xgb = random_search_xgb.best_estimator_
print("Best Parameters for XGBRegressor:", random_search_xgb.best_params_)

# Initialize base models for stacking with tuned hyperparameters
base_models = [
    ('random_forest', best_rf),
    ('xgboost', best_xgb),
    ('linear_regression', LinearRegression())
]

# Create a multivariate Stacking Regressor
stacked_model = StackingRegressor(estimators=base_models, final_estimator=Ridge(alpha=0.1))

# Use MultiOutputRegressor to handle multiple target variables
multi_output_model = MultiOutputRegressor(stacked_model)

# Fit the model to the training data
multi_output_model.fit(X_train_scaled, y_train)

# Save the multi-output model for later use
joblib.dump(multi_output_model, 'models/stacked_multivariate_model.pkl')

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END bootstrap=True, max_depth=15, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=15, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.6s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END bootstrap=True, max_depth=15, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.7s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   1.0s
[CV] END bootstrap=False, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time

['models/stacked_multivariate_model.pkl']

### Train & Testset: Prediction and Evaluation 

In [6]:
# Prediction on scaled training data
y_train_pred = multi_output_model.predict(X_train_scaled)

# Evaluate the model on the training set
train_mse = mean_squared_error(y_train, y_train_pred, multioutput='raw_values')
# train_rmse = np.sqrt(train_mse)
# train_mae = mean_absolute_error(y_train, y_train_pred, multioutput='raw_values')
train_r2 = r2_score(y_train, y_train_pred, multioutput='raw_values')
n_train = X_train_scaled.shape[0]
p_train = X_train_scaled.shape[1]
train_adj_r2 = 1 - (1 - train_r2) * ((n_train - 1) / (n_train - p_train - 1))
    
# Prediction on scaled test data
y_test_pred = multi_output_model.predict(X_test_scaled)

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test, y_test_pred, multioutput='raw_values')
# test_rmse = np.sqrt(test_mse)
# test_mae = mean_absolute_error(y_test, y_test_pred, multioutput='raw_values')
test_r2 = r2_score(y_test, y_test_pred, multioutput='raw_values')
n_test = X_test_scaled.shape[0]
p_test = X_test_scaled.shape[1]
test_adj_r2 = 1 - (1 - test_r2) * ((n_test - 1) / (n_test - p_test - 1))

# Print evaluation metrics for each target
for i, target in enumerate(targets):
    print(f"\nMetrics for {target}:")
    print(f"Train MSE: {train_mse[i]:.4f}")
    print(f"Test MSE: {test_mse[i]:.4f}")
    print(f"Train R^2: {train_r2[i]:.4f}")
    print(f"Test R^2: {test_r2[i]:.4f}")

# Calculate combined R² and Adjusted R² for training data
combined_train_r2 = r2_score(y_train, y_train_pred)
adjusted_r2_train_combined = 1 - (1 - combined_train_r2) * ((n_train - 1) / (n_train - p_train - 1))

# Calculate combined R² and Adjusted R² for test data
combined_test_r2 = r2_score(y_test, y_test_pred)
adjusted_r2_test_combined = 1 - (1 - combined_test_r2) * ((n_test - 1) / (n_test - p_test - 1))

# Print combined evaluation metrics for all target variables
print("\nCombined Evaluation Metrics for All Target Variables:")
print(f"Combined R² (Train): {combined_train_r2:.4f}")
print(f"Combined Adjusted R² (Train): {adjusted_r2_train_combined:.4f}")
print(f"Combined R² (Test): {combined_test_r2:.4f}")
print(f"Combined Adjusted R² (Test): {adjusted_r2_test_combined:.4f}")

# Print individual evaluation metrics for each target in a table format
headers = [
    "Target", "MSE (Train)", "MSE (Test)", "R^2 (Train)", "R^2 (Test)", "Adjusted R^2 (Train)", "Adjusted R^2 (Test)"
]
rows = []

for i, target in enumerate(targets):
    rows.append([
        target,
        train_mse[i],
        test_mse[i],
        train_r2[i],
        test_r2[i],
        train_adj_r2[i],
        test_adj_r2[i]
    ])

# Print the individual evaluation metrics in a table format
print("\n" + tabulate(rows, headers=headers, tablefmt="simple"))


Metrics for windpower:
Train MSE: 1528.0727
Test MSE: 5585.7264
Train R^2: 0.9752
Test R^2: 0.9013

Metrics for solar_pv:
Train MSE: 64.4663
Test MSE: 497.1368
Train R^2: 0.9941
Test R^2: 0.9559

Combined Evaluation Metrics for All Target Variables:
Combined R² (Train): 0.9847
Combined Adjusted R² (Train): 0.9844
Combined R² (Test): 0.9286
Combined Adjusted R² (Test): 0.9234

Target       MSE (Train)    MSE (Test)    R^2 (Train)    R^2 (Test)    Adjusted R^2 (Train)    Adjusted R^2 (Test)
---------  -------------  ------------  -------------  ------------  ----------------------  ---------------------
windpower      1528.07        5585.73        0.975183      0.901308                0.97475                0.894051
solar_pv         64.4663       497.137       0.994143      0.955899                0.994041               0.952656
